## Import Libraries and Data

In [154]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Specify dtype option on import or set low_memory=False.
pd.options.mode.chained_assignment = None  # default='warn'

import math
import scipy

# Datviz purposes
import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as px

In [155]:
try :
    train_data = pd.read_csv("/kaggle/input/penyisihan-joints-data-competition-2023/train.csv", low_memory=False)
    test_data = pd.read_csv("/kaggle/input/penyisihan-joints-data-competition-2023/test.csv", low_memory=False) # For Kaggle

except :
    train_data = pd.read_csv("../datasets/train.csv", low_memory=False)
    test_data = pd.read_csv("../datasets/test.csv", low_memory=False) # For local development

In [156]:
# Print all unique values in each column
for col in train_data.columns:
    print(col, train_data[col].unique())

print("\nTrain data shape: ", train_data.shape)

Unnamed: 0 [     0      1      2 ... 520008 520009 520010]
floors_before_eq (total) ['floor two' 'Floor 3' 'Two Floor' 'two' ' just 2 floor' nan
 'There is 2 Floor/Story' 'Floor two' '1st Story' 'Floor 2' 'fifth'
 'Three floor' 'floor one' '2' 'Has 1 floor' 'one story' 'two story'
 'floor second' 'floor third' '3.00' '1' ' has 3 Floor' '2 floor'
 'three Story' 'Floor 1' 'floor 1st' 'one' 'Just 1 floor' 'Floor-three'
 'Three' 'four Floor' ' has Four fl' 'Fl four' 'Fl Five' '6' '5' '4'
 'Floor 4' 'floor four' 'Four Story' 'Four' 'Floor Fifth' 'five Floor'
 ' Has Five fl' '8' '9' '7' 'Has Five fl']
old_building [  1.   3.   7.  18.  22.  nan  33.  40.  27.  68.  25.  15.  20.  28.
  35.  10.   4.   8.   5.  12.  32.  45.  13.  50.   6.  36.  70.  30.
  14.  75.  90.  34.  24.   2.   9.  16.  29. 999.  67.  60.  26.  38.
   0.  17.  37.  19.  80.  21.  11.  55.  44.  39.  23.  65.  48.  85.
  52.  31. 100.  57.  42.  41.  62. 120.  64.  43.  49.  47.  46.  88.
  82.  87.  69. 105.  99.  54

In [157]:
# Print all unique values in each column
for col in test_data.columns:
    print(col, test_data[col].unique())

print("\nTest data shape: ", test_data.shape)

id [     0      1      2 ... 242079 242080 242081]
floors_before_eq (total) ['2' '3.00' 'Floor-three' 'floor second' ' just 2 floor' 'Two Floor'
 ' has 3 Floor' 'There is 2 Floor/Story' 'two' 'Floor two' '2 floor'
 'Floor 1' 'three Story' 'two story' 'floor 1st' 'Floor 2' 'one story'
 'Three' 'floor one' 'five Floor' 'floor third' 'floor two' 'Floor 3' '1'
 'one' 'Three floor' 'Has 1 floor' 'Just 1 floor' '1st Story' 'Floor 4'
 'Four Story' 'Fl Five' ' has Four fl' '5' 'four Floor' 'Four' '6' '4'
 'floor four' ' Has Five fl' 'fifth' 'Fl four' 'Floor Fifth' '7' '9']
old_building [  7  13  40  25   5   6  10  17  31  18  44   2  15  16  12  30   3 999
  21   4  11   9  22  35  23  20  27   1   8  14  47  34  42  50 100  19
  32  33  64  28  60  45   0  29  70  26  38  41  39  54  24  37  80  43
  36  56  58  55  62 200  76  68  51  48  78  99  46  82  90  65 110 140
  52  75  63  57  85  83  81  61  53 114  71  95  72 102  94  69  74  73
  49 120 123  66  59 150 125  86  67 103  79 115  

## Data Preprocessing

In [158]:
train_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [159]:
train_data.head(1).transpose()

,0
floors_before_eq (total),floor two
old_building,1.0
plinth_area (ft^2),256 ft^2
height_before_eq (ft),22.0
land_surface_condition,Flat
type_of_foundation,Bamboo or Timber
type_of_roof,Bamboo/Timber Light roof
type_of_ground_floor,Clay
type_of_other_floor,TImber/Bamboo-Mud
position,Not attached


### lower case and replace spaces

In [160]:
# remove the space in column names
train_data.columns = train_data.columns.str.replace(' ', '')
test_data.columns = test_data.columns.str.replace(' ', '')

In [161]:
string_columns = ['floors_before_eq(total)','land_surface_condition','type_of_foundation','type_of_roof','type_of_ground_floor',
                  'type_of_other_floor','position','building_plan_configuration' ,'technical_solution_proposed',
                  'legal_ownership_status','residential_type','public_place_type','industrial_use_type',
                  'govermental_use_type','flexible_superstructure']

In [162]:
# Make every values in every column to lowercase
def make_lower_case(column):
    column = str(column)
    return column.lower()

for col in string_columns:
    train_data[col] = train_data[col].apply(make_lower_case)

for col in string_columns:
    test_data[col] = test_data[col].apply(make_lower_case)

### floors column to be an integer

In [163]:
one_floors = ['one', '1']
two_floors = ['two', '2', 'second']
three_floors = ['three', '3', 'third']
four_floors = ['four', '4']
five_floors = ['five', '5', 'fifth']
six_floors = ['six', '6']
seven_floors = ['seven', '7']
eight_floors = ['eight', '8']
nine_floors = ['nine', '9']

def handle_floors_before_eq(total):
    if any(word in total for word in one_floors):
        return 1
    elif any(word in total for word in two_floors):
        return 2
    elif any(word in total for word in three_floors):
        return 3
    elif any(word in total for word in four_floors):
        return 4
    elif any(word in total for word in five_floors):
        return 5
    elif any(word in total for word in six_floors):
        return 6
    elif any(word in total for word in seven_floors):
        return 7
    elif any(word in total for word in eight_floors):
        return 8
    elif any(word in total for word in nine_floors):
        return 9
    else:
        return total

train_data['floors_before_eq(total)'] = train_data['floors_before_eq(total)'].apply(handle_floors_before_eq)
test_data['floors_before_eq(total)'] = test_data['floors_before_eq(total)'].apply(handle_floors_before_eq)

### 'plinth_area_sq_ft'

In [164]:
def handle_area(area):
    area = str(area)
    area = area.split(' ')[0]
    return area

train_data['plinth_area(ft^2)'] = train_data['plinth_area(ft^2)'].apply(handle_area)
test_data['plinth_area(ft^2)'] = test_data['plinth_area(ft^2)'].apply(handle_area)

### 'legal_ownership_status'

In [165]:
private_values = ['private use','private','prvt','privste']
public_values = ['public space','public','public use']
institutional_values = ['institutionals','institutional use','institutional']
nan_values = ['nan','unknown','unspecified']

def handle_legal_ownership_status(status):
    if status in private_values:
        return 'private'
    elif status in public_values:
        return 'public'
    elif status in institutional_values:
        return 'institutional'
    elif status in nan_values:
        return 'nan'
    else:
        return status

train_data['legal_ownership_status'] = train_data['legal_ownership_status'].apply(handle_legal_ownership_status)
test_data['legal_ownership_status'] = test_data['legal_ownership_status'].apply(handle_legal_ownership_status)

### 'type_of_foundation'

In [166]:
def make_eda_using_correlation_within_damagegrade(df, cols):
    correlate = pd.DataFrame(
        index=[i for i in range(0, df[cols].nunique())],
        columns = df[cols].unique()
    )
    for index in correlate.index:
        for col in correlate.columns:
            correlate[col][index] = np.round(sum((df[cols] == col)&(df['damage_grade'] == index+1))/sum(df[cols] == col)*100, 2)

    # Change the index from 0,1,2,3,4 to 1,2,3,4,5
    correlate.index = [i+1 for i in range(0, correlate.shape[0])]
    # cut off the row index more than 5
    correlate = correlate.iloc[:5, :]

    return correlate

In [167]:
make_eda_using_correlation_within_damagegrade(train_data, 'type_of_foundation')

,bamboo or timber,clay sand mixed mortar-stone/brick,mud mortar-stone/brick,nan,clay mortar-stone/brick,reinforced concrete,cement-stone or cement-brick,bamboo/timber,rc,others,cement-stone/brick,other
1,30.52,4.73,4.78,10.28,4.64,58.66,29.52,29.6,59.61,29.45,30.29,28.66
2,23.71,8.86,9.01,11.53,8.95,27.21,22.61,23.01,26.93,9.27,22.17,9.15
3,22.41,17.91,17.86,17.7,18.03,10.17,18.5,23.2,9.82,10.91,18.26,10.3
4,14.22,26.67,26.77,24.19,26.87,2.2,14.03,14.95,2.19,15.18,13.82,15.05
5,9.14,41.82,41.59,36.3,41.5,1.77,15.35,9.23,1.45,35.19,15.45,36.85


In [168]:
bambo_or_timber = ['bamboo or timber','bamboo/timber']
mixed =  ['clay sand mixed mortar-stone/brick', 'mud mortar-stone/brick','clay mortar-stone/brick']
other = ['others','other']
rc = ['rc', 'reinforced concrete']
cement_stone_or_cement_brick = ['cement-stone or cement-brick','cement-stone/brick']

def handle_type_of_foundation(foundation):
    if foundation in bambo_or_timber:
        return 'bamboo_or_timber'
    elif foundation in other:
        return 'other'
    elif foundation in mixed:
        return 'mixed'
    elif foundation in rc:
        return 'rc'
    elif foundation in cement_stone_or_cement_brick:
        return 'cement_stone_or_cement_brick'
    else:
        return foundation

train_data['type_of_foundation'] = train_data['type_of_foundation'].apply(handle_type_of_foundation)
test_data['type_of_foundation'] = test_data['type_of_foundation'].apply(handle_type_of_foundation)

### 'type_of_roof'

In [169]:
make_eda_using_correlation_within_damagegrade(train_data, 'type_of_roof')

,bamboo/timber light roof,wood light roof or bamboo heavy roof,nan,wood light roof or bamboo light roof,bamboo or timber light roof,bamboo/timber-light roof,reinforced brick slab/rcc/rbc,bamboo/timber heavy roof,reinforced cement concrete/rb/rbc,bamboo or timber heavy roof,bamboo/timber-heavy roof,rcc/rb/rbc,reinforced brick concrete/rcc/rbc
1,8.17,6.38,10.27,8.3,7.99,8.14,55.06,6.19,54.7,6.62,6.2,55.45,52.96
2,10.36,10.42,11.56,10.38,10.99,10.02,28.36,10.06,27.49,9.18,10.89,27.05,29.17
3,17.88,19.38,17.84,17.6,17.68,18.3,10.94,20.01,11.87,19.34,19.12,12.22,11.7
4,25.92,24.26,24.14,25.89,25.66,25.76,3.61,24.15,3.56,24.45,23.6,3.01,3.69
5,37.68,39.57,36.19,37.82,37.67,37.78,2.03,39.59,2.38,40.42,40.19,2.28,2.48


In [170]:
bambo_or_timber_light = ['bamboo/timber light roof','bamboo or timber light roof',
                         'bamboo/timber-light roof','wood light roof or bamboo light roof']
bambo_or_timber_heavy = ['bamboo/timber heavy roof','bamboo or timber heavy roof','bamboo/timber-heavy roof',
                         'wood light roof or bamboo heavy roof']
rcc_rb_rbc = ['reinforced cement concrete/rb/rbc','rcc/rb/rbc', 'rcc_rbc', 
              'reinforced brick slab/rcc/rbc', 'reinforced brick concrete/rcc/rbc']

def handle_type_of_roof(roof):
    if roof in bambo_or_timber_light:
        return 'bamboo_or_timber_light'
    elif roof in bambo_or_timber_heavy:
        return 'bamboo_or_timber_heavy'
    elif roof in rcc_rb_rbc:
        return 'rcc_rb_rbc'
    else:
        return roof
    
train_data['type_of_roof'] = train_data['type_of_roof'].apply(handle_type_of_roof)
test_data['type_of_roof'] = test_data['type_of_roof'].apply(handle_type_of_roof)

### 'type_of_ground_floor'

In [171]:
make_eda_using_correlation_within_damagegrade(train_data, 'type_of_ground_floor')

,clay,nan,mud,brick or stone,reinforced concrete,rc,"soil, water, loam mixed",brick/stone,other,lumber,timber,wood
1,6.19,10.25,6.24,9.09,46.51,45.84,6.4,8.51,26.56,18.93,19.21,19.1
2,9.55,11.58,9.58,10.3,27.14,27.91,10.44,9.85,17.71,18.0,14.69,15.73
3,18.55,17.8,18.23,16.35,14.89,14.63,17.86,16.83,23.78,15.47,16.95,12.36
4,26.14,24.13,26.17,24.63,6.64,6.8,26.21,25.01,17.88,20.67,18.08,19.1
5,39.56,36.24,39.79,39.63,4.82,4.81,39.09,39.8,14.06,26.93,31.07,33.71


In [172]:
brick_or_stone = ['brick/stone','brick or stone']
rc = ['rc','reinforced cement', 'reinforced concrete']
wood = ['timber','lumber']
clay_mud = ['clay', 'mud', 'soil, water, loam mixed']

def handle_type_of_ground_floor(floor):
    if floor in brick_or_stone:
        return 'brick_or_stone'
    elif floor in rc:
        return 'rc'
    elif floor in wood:
        return 'wood'
    elif floor in clay_mud:
        return 'clay_mud'
    else:
        return floor
    
train_data['type_of_ground_floor'] = train_data['type_of_ground_floor'].apply(handle_type_of_ground_floor)
test_data['type_of_ground_floor'] = test_data['type_of_ground_floor'].apply(handle_type_of_ground_floor)

### 'type_of_other_floor'

In [173]:
make_eda_using_correlation_within_damagegrade(train_data, 'type_of_other_floor')

,timber/bamboo-mud,wood-mud or bamboo mud,timber mud or bamboo-mud,nan,not applicable,wood or bamboo mud,rcc/rb/rbc,wood-plank,timber-planck,lumber-plank,reinforced brick concrete/rcc/rbc,reinforced cement concrete/rb/rbc
1,4.11,4.32,4.06,10.22,25.9,4.07,51.43,9.32,9.33,9.35,53.42,50.5
2,8.92,8.96,8.84,11.57,17.72,9.51,27.51,11.3,11.16,10.64,27.4,27.65
3,19.15,19.05,19.01,17.81,15.38,19.05,12.96,16.65,17.03,17.02,12.88,12.71
4,28.73,28.98,28.79,24.16,12.92,29.5,4.59,21.74,21.07,22.01,3.7,5.35
5,39.09,38.69,39.3,36.24,28.09,37.87,3.51,40.99,41.41,40.98,2.6,3.79


In [174]:
wood_or_bambo_mud = ['wood-mud or bamboo mud','wood or bamboo mud','timber/bamboo-mud','timber mud or bamboo-mud']
rcc_rb_rbc = ['rcc/rb/rbc', 'reinforced cement concrete/rb/rbc', 'reinforced brick concrete/rcc/rbc']
wood_plank = ['wood-plank','timber-planck','lumber-plank']

def handle_type_of_other_floor(floor):
    if floor in wood_or_bambo_mud:
        return 'wood_or_bambo_mud'
    elif floor in rcc_rb_rbc:
        return 'rcc_rb_rbc'
    elif floor in wood_plank:
        return 'wood_plank'
    else:
        return floor

train_data['type_of_other_floor'] = train_data['type_of_other_floor'].apply(handle_type_of_other_floor)
test_data['type_of_other_floor'] = test_data['type_of_other_floor'].apply(handle_type_of_other_floor)

### nan -> np.nan

In [175]:
# Change 'nan' to np.nan
train_data = train_data.replace('nan', np.nan)
test_data = test_data.replace('nan', np.nan)

### 'plinth_area(ft^2)'

In [176]:
train_data[train_data['plinth_area(ft^2)'] == 'More'].shape

(6242, 24)

In [177]:
train_data['ismorethanplintharea'] = train_data['plinth_area(ft^2)'].apply(lambda x: 1 if x == 'More' else 0)
test_data['ismorethanplintharea'] = test_data['plinth_area(ft^2)'].apply(lambda x: 1 if x == 'More' else 0)

In [178]:
# Change 'more' to 0
train_data['plinth_area(ft^2)'] = train_data['plinth_area(ft^2)'].apply(lambda x: -1 if x == 'More' else float(x))
test_data['plinth_area(ft^2)'] = test_data['plinth_area(ft^2)'].apply(lambda x: -1 if x == 'More' else float(x))

### 'no_family_residing'

In [179]:
def handle_no_family_residing(no_family):
    if no_family == 'None':
        return 0
    elif no_family == np.nan:
        return np.nan
    else:
        return float(no_family)

train_data['no_family_residing'] = train_data['no_family_residing'].apply(handle_no_family_residing)
test_data['no_family_residing'] = test_data['no_family_residing'].apply(handle_no_family_residing)

### 'public_place_type'

In [180]:
public_places = ['shopping sites','recreational park','museum']
education_places = ['high school','university','middle school','primary school']
health_places = ['drugstore','health clinics','hospital']

def handle_public_place_type(public_place):
    if public_place in public_places:
        return 'public_places'
    elif public_place in education_places:
        return 'education_places'
    elif public_place in health_places:
        return 'health_places'
    else:
        return public_place

train_data['public_place_type'] = train_data['public_place_type'].apply(handle_public_place_type)
test_data['public_place_type'] = test_data['public_place_type'].apply(handle_public_place_type)

### 'position'

In [181]:
def handle_position(position):
    if position == 'not attached':
        return 0
    elif position == 'attached-1 side':
        return 1
    elif position == 'attached-2 side':
        return 2
    elif position == 'attached-3 side':
        return 3
    else:
        return position

train_data['position'] = train_data['position'].apply(handle_position)
test_data['position'] = test_data['position'].apply(handle_position)

### 'flexible_superstructure'

In [182]:
def handle_flexible_superstructure(flexible):
    if flexible == 'unavailable':
        return 0
    elif flexible == 'available':
        return 1
    else:
        return flexible

train_data['flexible_superstructure'] = train_data['flexible_superstructure'].apply(handle_flexible_superstructure)
test_data['flexible_superstructure'] = test_data['flexible_superstructure'].apply(handle_flexible_superstructure)

In [183]:
# Drop duplicated rows
print("---------before drop duplicated rows---------")
print("Shape of train data: ", train_data.shape)
print(f"Duplicated rows: {train_data.duplicated().sum()}")
train_data = train_data.drop_duplicates()

print("---------after drop duplicated rows---------")
print(f"Dropped duplicated rows: {train_data.duplicated().sum()}")
print("Shape of train data: ", train_data.shape)

---------before drop duplicated rows---------
Shape of train data:  (722815, 25)
Duplicated rows: 373813
---------after drop duplicated rows---------
Dropped duplicated rows: 0
Shape of train data:  (349002, 25)


In [184]:
# Print all unique values in each column
for col in train_data.columns:
    print(col, train_data[col].unique())

for col in test_data.columns:
    print(col, test_data[col].unique())

floors_before_eq(total) [ 2.  3. nan  1.  5.  4.  6.  8.  9.  7.]
old_building [  1.   3.   7.  18.  22.  nan  33.  40.  27.  68.  25.  15.  20.  28.
  35.  10.   4.   8.   5.  12.  32.  45.  13.  50.   6.  36.  70.  30.
  14.  75.  90.  34.  24.   2.   9.  16.  29. 999.  67.  60.  26.  38.
   0.  17.  37.  19.  80.  21.  11.  55.  44.  39.  23.  65.  48.  85.
  52.  31. 100.  57.  42.  41.  62. 120.  64.  43.  49.  47.  46.  88.
  82.  87.  69. 105.  99.  54. 140.  51.  72.  95.  63. 150. 176.  59.
  81.  58.  61.  53.  56.  84. 110.  66.  74.  98.  86. 109. 111. 200.
  73. 103.  79. 112.  78.  97.  92. 160.  71. 180.  76.  91.  96. 101.
 166. 106. 118. 108.  83.  77. 145. 130. 117.  93. 102. 135. 104. 170.
 131. 115.  89. 125. 128. 119.  94. 190. 174. 126. 132. 113. 116. 144.
 162. 138. 195. 141. 122. 175. 161. 196. 107. 121. 151. 142. 177. 124.
 127. 134. 178. 133. 123. 172. 168.]
plinth_area(ft^2) [ 256.  985.   nan  185.  290.  504.  616.  600.  500.   -1.  366.  150.
  300.  352.

In [185]:
df = pd.DataFrame(train_data.dtypes, columns=['Data Type'])
df['nunique'] = train_data.nunique()
df

,Data Type,nunique
floors_before_eq(total),float64,9
old_building,float64,160
plinth_area(ft^2),float64,930
height_before_eq(ft),float64,77
land_surface_condition,object,3
type_of_foundation,object,5
type_of_roof,object,3
type_of_ground_floor,object,5
type_of_other_floor,object,4
position,float64,4


In [186]:
df = pd.DataFrame(test_data.dtypes, columns=['Data Type'])
df['nunique'] = test_data.nunique()
df

,Data Type,nunique
id,int64,242082
floors_before_eq(total),int64,8
old_building,int64,144
plinth_area(ft^2),float64,928
height_before_eq(ft),int64,71
land_surface_condition,object,3
type_of_foundation,object,5
type_of_roof,object,3
type_of_ground_floor,object,5
type_of_other_floor,object,4


## Export the cleaned data to csv file

In [187]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)